# Filling the database
In this notebook we fill the database with three tables, one table that holds the actual routes, and two tables which serve as helpers, listing up gyms and rock climbing holds.

So after running this our database will have these tables:

```plaintext 
           / holds      - Lists the types of rock climbing holds 
          // gyms       - Lists gyms, with details like address, zip code, name etc.
climbing --- walls      - Lists the actual rock climbing walls, listing ratings, difficulties, the gym...
          \\ wall_holds - Junction table, connects holds to walls listing the quantity 
           \ users      - Lists users with favorite hold, location gym ...
```


## Imports and presetup

In [1]:
import pyodbc

import pandas as pd 
# While pandas has a builtin read_sql and read_sql_query method, it is meant for use with SQLAlchemy and would throw warnings
# So we need to write our own quick helper func
def query_to_df(query, cursor):
    cursor.execute(query)
    cols = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    df = pd.DataFrame.from_records(data = rows, columns=cols)
    df.set_index(cols[0], inplace = True)
    return df

from dotenv import load_dotenv
import os

## Connect to the SQL database using credentials from .env, and reset the database for a clean slate

In [ ]:
load_dotenv()
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

if not 'cnxn' in locals():
    cnxn = pyodbc.connect("DRIVER={MySQL ODBC 9.3 Unicode Driver};"
                        "SERVER=localhost;" 
                        f"DATABASE={DB_NAME};"
                        f"USER={USER};"
                        f"PASSWORD={PASSWORD};"
                        "PORT=3306;"
                        "OPTION=3;")

cnxn.cursor().execute("DROP TABLE IF EXISTS users;")
cnxn.cursor().execute("DROP TABLE IF EXISTS wall_holds;")
cnxn.cursor().execute("DROP TABLE IF EXISTS walls;")
cnxn.cursor().execute("DROP TABLE IF EXISTS holds;")
cnxn.cursor().execute("DROP TABLE IF EXISTS gyms;")

cnxn.commit()

cursor = cnxn.cursor()

## Fill the holds table with preset hold names

In [3]:
# Create a table listing hold types by autogenerated ids
cursor.execute("""
                CREATE TABLE holds (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    name VARCHAR(255) UNIQUE NOT NULL
               );
               """)
cnxn.commit()

# Fill it with some common hold names
cursor.execute("""
                INSERT INTO holds (name)
                VALUES 
                    ('Jug'),
                    ('Edge'),
                    ('Slab'),
                    ('Crimp'),
                    ('Pinch'),
                    ('Sloper'),
                    ('Undercling')
               """)
cnxn.commit()

# Run a selection to check that we've got what we expect in the table
temp_df = query_to_df("SELECT * FROM holds ORDER BY id", cursor)
temp_df.head()


,name
id,
1,Jug
2,Edge
3,Slab
4,Crimp
5,Pinch


## Create a randomized list of gyms

### Create a random list of gyms

In [4]:
import random

# Set up a list of states and associated cities to pull from 
states = ['Pennsylvania', 'New York', 'New Jersey']
cities = [['Philadelphia', 'Harrisburg', 'Pittsburgh'],['New York', 'Newark'],['Wilmington','Glassboro','Atlantic City']]
ZIP_Prefix = [[150,196],[10,14],[7,8]]

# Set up a list of randomized street name components
Street_init = ['Jane', 'Cherry', 'Dr Watson', 'Colonial', 'Center', 'Duck', 'Swan', 'New']
Street_terminal = [' Lane',' Street',' Boulevard',' Industrial Park', ' Drive']
Street_num = [0,350]

# Set up a list of randomized gym name components
Gym_prefix = ['Dave’s', 'Horton’s', 'Clear', 'Big', 'Climbathon']
Gym_terminal = [' Gym', ' Climb', ' Climbing Gym', ' Wall House']

# Generate every possible gym name from the above
names = [a + b for a in Gym_prefix for b in Gym_terminal]
gyms = []

# Associate each gym name with a random adress, city, state etc.
for name in names:
    state_idx = random.randint(0,len(states)-1) 
    state = states[state_idx]

    city = random.choice(cities[state_idx])

    zip_p = f"{random.randint(ZIP_Prefix[state_idx][0], ZIP_Prefix[state_idx][1]):03d}"
    zip_e = f"{random.randint(0,99):02d}"
    zip = zip_p + zip_e
    
    street = random.choice(Street_init) + random.choice(Street_terminal) + ' ' + str(random.randint(Street_num[0], Street_num[1]))
    
    gyms.append([name,state,city,zip,street])

random.shuffle(gyms) # Shuffle the gym list to keep all the Dave's from clumping 

del(zip)

### Create a table and fill it with the randomized gyms

In [5]:
# Create a table listing gyms that can hold all the data we wanted to add
cursor.execute("""
                CREATE TABLE gyms (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    Name VARCHAR(255) UNIQUE NOT NULL,
                    State VARCHAR(255) NOT NULL,
                    City VARCHAR(255) NOT NULL,
                    ZipCode VARCHAR(255) NOT NULL,
                    Address VARCHAR(255) NOT NULL
               );
               """)
cnxn.commit()

# Put together a dynamic insertion string that 
insert_root = """
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
"""

insert = ''

for gym in gyms:
    insertion_str = ''
    for ele in gym:
        insertion_str = insertion_str + '\'' + ele + '\'' + ','

    insert = insert + '    (' + insertion_str[0:-1] + '),' + '\n'

insert = insert_root + insert[0:-2]+ ';'

# Let's show what the somewhat messily constructed insertion string looks like
print('Insertion: ', end='\n---------------------')
print(insert)

# And Run it
cnxn.execute(insert)
cnxn.commit()

# Then just like before let's check that the insertion did what we expected
print('\n\nResult:')
print('---------------------')
temp_df = query_to_df("SELECT * FROM gyms ORDER BY id", cursor)
temp_df.head()

Insertion: 
---------------------
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
    ('Climbathon Wall House','New York','New York','01090','Center Industrial Park 145'),
    ('Big Climb','New York','Newark','01248','New Street 219'),
    ('Horton’s Climb','Pennsylvania','Harrisburg','18307','Swan Street 50'),
    ('Big Climbing Gym','Pennsylvania','Pittsburgh','16816','Cherry Drive 176'),
    ('Big Wall House','New York','New York','01465','New Boulevard 103'),
    ('Dave’s Wall House','New Jersey','Glassboro','00754','Cherry Boulevard 326'),
    ('Horton’s Climbing Gym','New Jersey','Glassboro','00876','New Boulevard 193'),
    ('Climbathon Gym','New York','Newark','01493','Jane Boulevard 28'),
    ('Dave’s Climb','Pennsylvania','Harrisburg','15181','Center Lane 17'),
    ('Climbathon Climb','Pennsylvania','Philadelphia','17738','Cherry Boulevard 1'),
    ('Big Gym','New York','Newark','01228','Colonial Street 1'),
    ('Clear Climbing Gym','New Jersey','Wilmingto

,Name,State,City,ZipCode,Address
id,,,,,
1,Climbathon Wall House,New York,New York,01090,Center Industrial Park 145
2,Big Climb,New York,Newark,01248,New Street 219
3,Horton’s Climb,Pennsylvania,Harrisburg,18307,Swan Street 50
4,Big Climbing Gym,Pennsylvania,Pittsburgh,16816,Cherry Drive 176
5,Big Wall House,New York,New York,01465,New Boulevard 103


## Now we can finally make some climbing-walls and associate them with gyms

### Set up the wall table, and a junction table to connect walls to holds

In [6]:
# Set up a new table that stores walls and links them to their respective gyms.
cursor.execute("""
                CREATE TABLE walls (
                    id SMALLINT PRIMARY KEY AUTO_INCREMENT,
                    name VARCHAR(255), 
                    gym SMALLINT,
                    rating FLOAT,
                    num_ratings INT NOT NULL,
                    difficulty VARCHAR(255),
                    CHECK(rating >= 0 AND rating <= 5),
                    FOREIGN KEY (gym) REFERENCES gyms(id)
                );
               """)
cnxn.commit()

# Set up a new table that stores the holds present by wall (Junction / Graph)
cursor.execute("""
                CREATE TABLE wall_holds (
                    wall_id SMALLINT,
                    hold_id SMALLINT,
                    quantity SMALLINT NOT NULL DEFAULT 1,
                    FOREIGN KEY (hold_id) REFERENCES holds(id),
                    FOREIGN KEY (wall_id) REFERENCES walls(id),
                    PRIMARY KEY (wall_id, hold_id),
                    CHECK(quantity >= 1)
                );
               """)
cnxn.commit()


# ########################################## 
# Now we can demonstrate how a manuaL wall insertion would look before we automate it in the next cell

# Create a standardish, named, wall with a couple of ratings, a fair rating and a standard difficulty, and
# a wall in gym two that's marked as a bit harder, rated poorly, and nameless.
cursor.execute("""
                INSERT INTO walls (gym, name, rating, num_ratings, difficulty)
                VALUES
                    (1, 'Brave Blue', 3.5, 27, 'V6'),
                    (2, Null, 2.5, 1, 'V8')
               """)

# Associate some holds to the walls 
# Very vaguely wall 1 has a mix of holds, and wall two is a balancy wall with tons of slab and some crimp
cursor.execute("""
                INSERT INTO wall_holds (wall_id, hold_id, quantity)
                VALUES
                    (1, 1, 4),
                    (1, 2, 2),
                    (1, 7, 2),
                    (1, 3, 1),
                    (2, 4, 3),
                    (2, 3, 8)
               """)

# And display the result of our queries, but joined up so that we can read out the holds in a clear way.
get_walls = """
        SELECT 
               walls.id AS wall_id,
               gyms.name AS gym_name,
               walls.name AS name,
               walls.rating,
               walls.difficulty
        FROM walls
        INNER JOIN gyms ON walls.gym = gyms.id
        ORDER BY walls.rating DESC, gyms.name, walls.id
        """

print("Walls after creation of wall table:")
df = query_to_df(get_walls, cursor)
display(df.head())

# Or if we want to know how the holds are distributed
get_holds = """
        SELECT  
                holds.name AS hold,
                CONCAT(
                    COALESCE(walls.name, 'Not Named'),
                    ' : ',
                    walls.id
                ) AS 'wall name : id',
                wall_holds.quantity AS count
        FROM 
            wall_holds
        INNER JOIN 
            holds ON holds.id = wall_holds.hold_id
        INNER JOIN
            walls ON walls.id = wall_holds.wall_id
        ORDER BY 
            walls.rating DESC, wall_id, hold
        """

print("\n\nPresence of holds on walls")
df = query_to_df(get_holds, cursor)
display(df)

Walls after creation of wall table:


,gym_name,name,rating,difficulty
wall_id,,,,
1,Climbathon Wall House,Brave Blue,3.5,V6
2,Big Climb,None,2.5,V8




Presence of holds on walls


,wall name : id,count
hold,,
Edge,Brave Blue : 1,2
Jug,Brave Blue : 1,4
Slab,Brave Blue : 1,1
Undercling,Brave Blue : 1,2
Crimp,Not Named : 2,3
Slab,Not Named : 2,8


### Randomly Generate a bunch of walls

In [ ]:
# First we make some lists for the randomization to pull from
import urllib.request
words = urllib.request.urlopen("https://www.mit.edu/~ecprice/wordlist.10000").read().decode().splitlines() # https://stackoverflow.com/questions/18834636/random-word-generator-python
difficulties = ['B'] + [str(num) for num in range(0,13)]
holds = query_to_df("Select * From holds", cursor).index.tolist()

# Define a wall generation function that we can run for as many walls as we want
def generate_wall():
    score = random.random()*5
    num_ratings = random.randint(0,1000)
    
    difficulty = 'V' + str(random.choice(difficulties))

    nameL = random.sample(words, random.randint(0,2))
    name = '-'.join([str(a) for a in nameL])
    if len(name) == 0: name = None

    hold = random.sample(holds, random.randint(1,7))
    counts = [random.randint(1,9) for h in hold]

    gymid = random.randint(1,len(gyms))

    return ([gymid, name, score, num_ratings, difficulty], [hold, counts])

# Set An Average of 6 walls per gym and roll the wals
num_walls = len(gyms)*6 
walls = [generate_wall() for i in range(num_walls)]

# Loop over each of the walls we created in the list and make a query to insert it, as well as adding the new holds
for wall in walls:
    wall_info = wall[0]
    insert_str = f"""INSERT INTO walls (gym, name, rating, num_ratings, difficulty)
                      VALUES
                         ({wall_info[0]},'{wall_info[1]}',{wall_info[2]},{wall_info[3]},'{wall_info[4]}')"""
    cursor.execute(insert_str)

    cursor.execute("SELECT LAST_INSERT_ID()") # Need this to grab the id to associate the holds to since we're autogenning ids.
    id = cursor.fetchone()[0] #type: ignore

    # To add the holds we transform the paired list of hold ids and counts to a list of tuples before fstringing them into a request
    holds_info = wall[1]
    for hold_info in zip(holds_info[0], holds_info[1]): #type: ignore
            insert_str = f"""
                INSERT INTO wall_holds (wall_id, hold_id, quantity)
                VALUES
                    ({id},{hold_info[0]},{hold_info[1]})
               """
            cursor.execute(insert_str)

             
# Check the output
print("Walls after filling of wall table:")
df = query_to_df(get_walls, cursor)
display(df)

print("\n\nPresence of holds on walls")
df = query_to_df(get_holds, cursor)
display(df)

Walls after filling of wall table:


,gym_name,name,rating,difficulty
wall_id,,,,
42,Big Wall House,speaks,4.970270,V1
67,Climbathon Climb,None,4.911740,VB
73,Climbathon Gym,None,4.899110,VB
58,Clear Wall House,None,4.876880,V10
113,Horton’s Climb,None,4.816130,V7
...,...,...,...,...
101,Horton’s Climbing Gym,phys-conditioning,0.229863,V2
82,Clear Climbing Gym,None,0.223421,V12
26,Clear Climb,vg,0.148625,V12




Presence of holds on walls


,wall name : id,count
hold,,
Crimp,speaks : 42,5
Edge,speaks : 42,9
Jug,speaks : 42,4
Pinch,speaks : 42,8
Sloper,speaks : 42,1
...,...,...
Pinch,retrieved : 28,7
Sloper,retrieved : 28,8
Crimp,nursing : 43,8


## Now we add three dummy users manually that we can test from later

### Create the user table

In [ ]:
# Create a user table
cursor.execute("""
CREATE TABLE users (
    id SMALLINT AUTO_INCREMENT PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    favorite_hold SMALLINT,
    city VARCHAR(255),
    state VARCHAR(255),
    gym SMALLINT,
    FOREIGN KEY (gym) REFERENCES gyms(id),
    FOREIGN KEY (favorite_hold) REFERENCES holds(id)
)
""")
cursor.commit()

### Fill the user table

In [ ]:
# Let's start with a couple John and Jane who live in Pennsylvania and have filled in a favorite gym, Jane likes slab and John likes edge
# And let's add their son Jack Doe who's moved to New Jersey but has the same favorite gym as his parents
cursor.execute("""
INSERT INTO 
    users (email, favorite_hold, city, state, gym)
VALUES
    ("JaneDoe@goodData.com", 3, "Harrisburg", "Pennsylvania", 3),
    ("JohnDoe@goodData.com", 2, "Harrisburg", "Pennsylvania", 3),
    ("JackDoe@goodData.com", 6, "Cherry Hill", "New Jersey", 3)
""")

# Now lets create three users who are a bit less commited, one each who skipped filling out a favorite hold, city and state or gym
cursor.execute("""
INSERT INTO
    users (email, favorite_hold, city, state)
VALUES
    ("NoahJims@skipMail.org", 3, "Newark", "New York")
""")

cursor.execute("""
INSERT INTO
    users (email, city, state, gym)
VALUES
    ("HoldenLacksson@skipMail.org", "Newark", "New York", 10)
""")

cursor.execute("""
INSERT INTO
    users (email, favorite_hold, gym)
VALUES
    ("BenAway@skipMail.org", 4, 2)
""");


### Show the user table (Joined with Gym names and Hold names for clarity)

In [ ]:
# Put together and run query for users info.
get_users = """
SELECT 
    users.id AS uid,
    users.email AS email,
    gyms.name AS gym,
    holds.name AS 'favorite hold',
    CONCAT(users.city,'-', users.state) AS location
FROM users
LEFT JOIN gyms ON users.gym = gyms.id
LEFT JOIN holds ON users.favorite_hold = holds.id
"""
df = query_to_df(get_users, cursor)
display(df)


,email,gym,favorite hold,location
uid,,,,
1,JaneDoe@goodData.com,Horton’s Climb,Slab,Harrisburg-Pennsylvania
2,JohnDoe@goodData.com,Horton’s Climb,Edge,Harrisburg-Pennsylvania
3,JackDoe@goodData.com,Horton’s Climb,Sloper,Cherry Hill-New Jersey
4,NoahJims@skipMail.org,None,Slab,Newark-New York
5,HoldenLacksson@skipMail.org,Climbathon Climb,None,Newark-New York
6,BenAway@skipMail.org,Big Climb,Crimp,None


## With that our DB is filled so the next step is to set up an interface and some queries to run in another file

### So let's close up our connection

In [19]:
# Close cursor
cursor.close()
cnxn.close()